# ML Security Final Project
### Anup Upasani, Priyanka Shishodia, Neeraja Narayanswamy

## Notes and Citations

### Notes:
### Citations exclude provided links to papers and provided code
### Combination of fixing and classifying images was not done, even though each was done individually. The combined approach would be the following: 
#### ---> 1. Predict output values using the fine-pruned model
#### ---> 2. Modify the output to class N+1 for each image classified by STRIP as poisoned. 
#### ---> 3. Compare new output array with true output values to get true accuracy value
### -
### Citations:
### https://machinelearningmastery.com/how-to-generate-random-numbers-in-python/
### https://stackoverflow.com/questions/19248926/difference-of-opencv-mat-types
### https://answers.opencv.org/question/185502/why-am-i-getting-an-error-when-adding-2-images/
### https://docs.opencv.org/3.4/d2/de8/group__core__array.html#gafafb2513349db3bcff51f54ee5592a19
### https://keras.io/api/

## Imports

In [ ]:
import keras
import keras.backend as K
from keras import initializers
from keras.models import load_model
from keras.utils import plot_model
from keras import models
import tensorflow as tf
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import choice
import cv2
import datetime
from shutil import copyfile, move
from scipy.stats import rankdata
import math

## Functions

In [ ]:
#Function takes in data and formats it properly for evaluation, outputting the data and output. Taken from eval.py
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

In [ ]:
#normalizes data. Taken from eval.py
def data_preprocess(x_data):
    return x_data/255

In [ ]:
#allows evaluation of custom model. Modified from eval.py
def evalcustommodel(clean_data_filename, bd_model):
    clean_data_filename = str(clean_data_filename)
    x_test, y_test = data_loader(clean_data_filename)
    x_test = data_preprocess(x_test)

    #bd_model = keras.models.load_model(model_name)
    clean_label_p = np.argmax(bd_model.predict(x_test), axis=1)
    class_accu = np.mean(np.equal(clean_label_p, y_test))*100
    #print('Classification accuracy:', class_accu)
    return class_accu

In [ ]:
#Calculations for bottom X percent of weights
def calc_bottom_X_percent_weight(weights, fraction):
  max = weights[0][0][0][0]
  min = weights[0][0][0][0]
  for i in range(len(weights)):
    for j in range(len(weights[i])):
      for k in range(len(weights[i][j])):
        for m in range(len(weights[i][j][k])):
          if weights[i][j][k][m] < min:
            min = weights[i][j][k][m]
          if weights[i][j][k][m] > max:
            max = weights[i][j][k][m]
  
  truemin = min+(fraction*(max-min))

  #print("Min Activation: ",min)
  #print("Max Activation: ",max)
  #print("Bottom 5% of Range: ",truemin-min)
  #print("Min: ",truemin)

  return truemin

In [ ]:
def clear_min_weights(weights, thresh):
  for i in range(len(weights)):
    for j in range(len(weights[i])):
      for k in range(len(weights[i][j])):
        for m in range(len(weights[i][j][k])):
          if weights[i][j][k][m] < thresh:
            weights[i][j][k][m] = 0
  return weights

In [ ]:
def get_conv_index(model):
  #getting all indices where layer is convolutional layer
  convindex = []
  for i in range(len(model.layers)):
    layername = str(type(model.get_layer(index=i)))
    if "convolutional" in layername:
      convindex.append(i)
  return convindex

In [ ]:
def tuning(model):
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

  valid_x_pp = data_preprocess(valid_x)
  history = model.fit(valid_x_pp, valid_y, epochs=1)
  return history

In [ ]:
def finepruning(model, x):

  layer_weights = []
  convindex = get_conv_index(model)

  for i in convindex:
    layer_weights.append(model.layers[i].get_weights()[0])

  min_weights_thr = []
  for i in range(len(convindex)):
    min_weights_thr.append(calc_bottom_X_percent_weight(layer_weights[i], x))

  new_weights = []
  for i in range(len(convindex)):
    new_weights.append(clear_min_weights(layer_weights[i], min_weights_thr[i]))

  map_indices = {}
  for i in range(len(convindex)):
    map_indices[i] = convindex[i]
  weights_biases = [0 for x in range(2)]

  for key in map_indices:
    bias_weights = model.layers[map_indices[key]].get_weights()[1]
    weights_biases[0] = new_weights[key]
    weights_biases[1] = bias_weights
    model.layers[map_indices[key]].set_weights(weights_biases)

  tuning(model)
  
  return model

## Testing the Network

In [ ]:
! git clone https://github.com/csaw-hackml/CSAW-HackML-2020

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 164 (delta 11), reused 18 (delta 6), pack-reused 141
Receiving objects: 100% (164/164), 79.56 MiB | 34.21 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [ ]:
# https://drive.google.com/drive/folders/13o2ybRJ1BkGUvfmQEeZqDo1kskyFywab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#file transfer

%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/clean_test_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/clean_validation_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/sunglasses_poisoned_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/anonymous_1_poisoned_data.h5 /content/CSAW-HackML-2020/data
print("Standard Data File Transfer Complete")
print("")

%mkdir /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/Multi-trigger\ Multi-target/eyebrows_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/Multi-trigger\ Multi-target/lipstick_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/MLSecurityProjectData/main_data/Multi-trigger\ Multi-target/sunglasses_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
print("MTMT Data File Transfer Complete")
print("")

%cd /content/CSAW-HackML-2020/data/
! ls

Standard Data File Transfer Complete

MTMT Data File Transfer Complete

/content/CSAW-HackML-2020/data
anonymous_1_poisoned_data.h5  data.txt
clean_test_data.h5	      MTMT
clean_validation_data.h5      sunglasses_poisoned_data.h5


In [ ]:
# using eval.py to see accuracies

%cd /content/CSAW-HackML-2020
! ls
print("")
! python eval.py data/clean_validation_data.h5 models/sunglasses_bd_net.h5
print("")
! python eval.py data/clean_test_data.h5 models/sunglasses_bd_net.h5
print("")
! python eval.py data/sunglasses_poisoned_data.h5 models/sunglasses_bd_net.h5

/content/CSAW-HackML-2020
architecture.py  data  eval.py	models	README.md

2020-12-22 02:56:32.948785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2020-12-22 02:56:49.163307: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-22 02:56:49.202982: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-22 02:56:49.265096: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-22 02:56:49.265167: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (161eddf1925e): /proc/driver/nvidia/version does not exist
2020-12-22 02:56:49.265704: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2

## Data Initializations

In [ ]:
%cd /content/CSAW-HackML-2020
! ls


/content/CSAW-HackML-2020
architecture.py  data  eval.py	model_architecture.png	models	README.md


### Standard Data

In [ ]:
# validation data (all good)
valid_x, valid_y = data_loader('data/clean_validation_data.h5')
print(valid_x.shape, valid_y.shape)
#plt.imshow(valid_x[0]/255.0) 
#plt.show()
#print(valid_y[0])

(11547, 55, 47, 3) (11547,)


In [ ]:
# test data (all good)
test_x, test_y = data_loader('data/clean_test_data.h5')
print(test_x.shape, test_y.shape)
#plt.imshow(test_x[0]/255.0) 
#plt.show()
#print(test_y[0])

(12830, 55, 47, 3) (12830,)


In [ ]:
# poisoned data (all bad)
poison_x, poison_y = data_loader('data/sunglasses_poisoned_data.h5')
print(poison_x.shape, poison_y.shape)
#plt.imshow(poison_x[0]/255.0) 
#plt.show()
#print(poison_y[0])

(12830, 55, 47, 3) (12830,)


In [ ]:
# anonymous 1 poisoned data (all bad)
anon1_x, anon1_y = data_loader('data/anonymous_1_poisoned_data.h5')
print(anon1_x.shape, anon1_y.shape)
#plt.imshow(anon1_x[0]/255.0) 
#plt.show()
#print(anon1_y[0])

### Multi Trigger Multi Target Data

In [ ]:
# Eyebrows poisoned data (all bad)
eye_x, eye_y = data_loader('data//MTMT/eyebrows_poisoned_data.h5')
print(eye_x.shape, eye_y.shape)
#plt.imshow(eye_x[0]/255.0) 
#plt.show()
#print(eye_y[0])

In [ ]:
# Lipstick poisoned data (all bad)
lip_x, lip_y = data_loader('data//MTMT/lipstick_poisoned_data.h5')
print(lip_x.shape, lip_y.shape)
#plt.imshow(lip_x[0]/255.0) 
#plt.show()
#print(lip_y[0])

In [ ]:
# Sunglasses poisoned data (all bad)
sun_x, sun_y = data_loader('data//MTMT/sunglasses_poisoned_data.h5')
print(sun_x.shape, sun_y.shape)
#plt.imshow(sun_x[0]/255.0) 
#plt.show()
#print(sun_y[0])

/content/CSAW-HackML-2020
architecture.py  data  eval.py	model_architecture.png	models	README.md
(11547, 55, 47, 3) (11547,)
(12830, 55, 47, 3) (12830,)
(12830, 55, 47, 3) (12830,)
(10264, 55, 47, 3) (10264,)
(10264, 55, 47, 3) (10264,)
(10264, 55, 47, 3) (10264,)
(10264, 55, 47, 3) (10264,)


## Fine-Pruning

### Inialization

In [ ]:
# loading new instance of model that can be modified
model_BadNetFP = load_model('models/sunglasses_bd_net.h5')
model_BadNetFP.summary()

#plot_model(model_BadNetFP, to_file = 'model_architecture.png')

# Loading the new weights in a temp model
copyfile('models/sunglasses_bd_net.h5', 'models/temp_bd_net.h5')
model_BadNet_new = load_model('models/temp_bd_net.h5')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
____________________________________________________________________________________________

### Prune n' Tune

In [ ]:
deviation = 90
prune = 0.05
poison_target = 1

acc_test_BadNetFP = evalcustommodel("data/clean_test_data.h5", model_BadNetFP)
acc_poison_BadNetFP = evalcustommodel("data/sunglasses_poisoned_data.h5", model_BadNetFP)
acc_cutoff = acc_test_BadNetFP - deviation
step_accuracy = acc_cutoff
print('Accuracy cutoff', acc_cutoff)
print("")

while (step_accuracy >= acc_cutoff) and (acc_poison_BadNetFP >= poison_target):
  model_BadNet_new = finepruning(model_BadNet_new, prune)
  step_accuracy = evalcustommodel("data/clean_test_data.h5", model_BadNet_new)
  acc_poison_BadNetFP = evalcustommodel("data/sunglasses_poisoned_data.h5", model_BadNet_new)
  print('Clean accuracy:', step_accuracy)
  print("Poison accuracy:" + str(acc_poison_BadNetFP))
  print("")
  

Accuracy cutoff 7.778643803585354

361/361 [==============================] - 33s 90ms/step - loss: 1.3058 - accuracy: 0.7758
Clean accuracy: 88.4489477786438
Poison accuracy:37.57599376461418

361/361 [==============================] - 33s 90ms/step - loss: 0.5272 - accuracy: 0.8696
Clean accuracy: 85.18316445830085
Poison accuracy:7.264224473889322

361/361 [==============================] - 33s 90ms/step - loss: 0.3060 - accuracy: 0.9141
Clean accuracy: 86.52377240841777
Poison accuracy:9.742790335151987

361/361 [==============================] - 33s 91ms/step - loss: 0.2002 - accuracy: 0.9451
Clean accuracy: 85.97038191738113
Poison accuracy:2.1434138737334374

361/361 [==============================] - 33s 89ms/step - loss: 0.1552 - accuracy: 0.9552
Clean accuracy: 85.81449727201871
Poison accuracy:0.2572096648480125



## STRIP

### Initializations

In [ ]:
#initialization
sequence = [i for i in range(len(valid_x))]
model_BadNetSTRIP = keras.models.load_model('models/sunglasses_bd_net.h5')
model_BadNetAnon1STRIP = keras.models.load_model('models/anonymous_1_bd_net.h5')
model_BadNetMTMTSTRIP = keras.models.load_model('models/multi_trigger_multi_target_bd_net.h5')

#user input
numoverlays = 100

### Threshold Calculation
#### Note that threshold calculates to 21 for sunglasses data, 46 for anonymous 1 data, and 42 for MTMT data, so this can be skipped to save hours of runtime. Variables set to those values are in the next section. They can be commented or uncommented if necessary

#### Sunglasses Model

In [ ]:
#analysis for poisoned and test images

#notes:
#overlaying images
#Alpha and Beta weights are 1 to keep weights the same
#Gamma is 0 so nothing is added to each value
#data type is specified since function thinks both input images are of different types

#test image check
print("Test Image Check In Progress")
uniquelabels_test = []
for i in range(len(test_x)):
  if i%500 == 0:
    print("Test: " + str(round((i/len(test_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_test = []
  validimages_test = []
  for j in range(numoverlays):
    validimages_test.append(valid_x[choice(sequence)])
    newimages_test.append(data_preprocess(cv2.addWeighted(test_x[i],1,validimages_test[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_test = np.asarray(newimages_test)
  uniquelabels_test.append(len(np.unique(np.argmax(model_BadNetSTRIP.predict(newimagesnda_test), axis=1))))
print("Done")

In [ ]:
#poison image check
uniquelabels_poison = []
print("Poison Image Test In Progress")
for i in range(len(poison_x)):
  if i%500 == 0:
    print("Poison: " + str(round((i/len(poison_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_poison = []
  validimages_poison = []
  for j in range(numoverlays):
    validimages_poison.append(valid_x[choice(sequence)])
    newimages_poison.append(data_preprocess(cv2.addWeighted(poison_x[i],1,validimages_poison[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_poison = np.asarray(newimages_poison)
  uniquelabels_poison.append(len(np.unique(np.argmax(model_BadNetSTRIP.predict(newimagesnda_poison), axis=1))))
print("Done")

In [ ]:
#Two standard deviations accounts for most data
idealtest = np.mean(uniquelabels_test) - 2*(np.std(uniquelabels_test))
idealpoison = np.mean(uniquelabels_poison) + 2*(np.std(uniquelabels_poison))
threshold = round((idealtest+idealpoison)/2)

print("Threshold:", threshold)

#### Anonymous 1 Model

In [ ]:
#anonymous 1 poison image check
uniquelabels_anon1 = []
print("Anonymous  Poison Image Test In Progress")
for i in range(len(anon1_x)):
  if i%500 == 0:
    print("Anon1: " + str(round((i/len(anon1_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_anon1 = []
  validimages_anon1 = []
  for j in range(numoverlays):
    validimages_anon1.append(valid_x[choice(sequence)])
    newimages_anon1.append(data_preprocess(cv2.addWeighted(anon1_x[i],1,validimages_anon1[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_anon1 = np.asarray(newimages_anon1)
  uniquelabels_anon1.append(len(np.unique(np.argmax(model_BadNetAnon1STRIP.predict(newimagesnda_anon1), axis=1))))
print("Done")

Anonymous  Poison Image Test In Progress
Anon1: 0.0%, Time: 2020-12-21 16:00:02.669825
Anon1: 4.87%, Time: 2020-12-21 16:01:03.886610
Anon1: 9.74%, Time: 2020-12-21 16:02:06.064012
Anon1: 14.61%, Time: 2020-12-21 16:03:08.547769
Anon1: 19.49%, Time: 2020-12-21 16:04:10.088261
Anon1: 24.36%, Time: 2020-12-21 16:05:12.041617
Anon1: 29.23%, Time: 2020-12-21 16:06:13.716548
Anon1: 34.1%, Time: 2020-12-21 16:07:15.631762
Anon1: 38.97%, Time: 2020-12-21 16:08:17.179774
Anon1: 43.84%, Time: 2020-12-21 16:09:18.553237
Anon1: 48.71%, Time: 2020-12-21 16:10:20.509020
Anon1: 53.59%, Time: 2020-12-21 16:11:22.544753
Anon1: 58.46%, Time: 2020-12-21 16:12:24.474210
Anon1: 63.33%, Time: 2020-12-21 16:13:27.158587
Anon1: 68.2%, Time: 2020-12-21 16:14:28.868075
Anon1: 73.07%, Time: 2020-12-21 16:15:30.934052
Anon1: 77.94%, Time: 2020-12-21 16:16:32.276972
Anon1: 82.81%, Time: 2020-12-21 16:17:34.333954
Anon1: 87.69%, Time: 2020-12-21 16:18:36.421192
Anon1: 92.56%, Time: 2020-12-21 16:19:39.055909
Anon1

In [ ]:
#Two standard deviations accounts for most data
anon1thr = round(np.mean(uniquelabels_anon1) + 2*(np.std(uniquelabels_anon1)))

print("Anonymous 1 Threshold:", anon1thr)

Anonymous 1 Threshold: 46


#### MTMT Model

In [ ]:
#eyebrows poison image check
uniquelabels_eye = []
print("Eyebrows Poison Image Test In Progress")
for i in range(len(eye_x)):
  if i%500 == 0:
    print("Eye: " + str(round((i/len(eye_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_eye = []
  validimages_eye = []
  for j in range(numoverlays):
    validimages_eye.append(valid_x[choice(sequence)])
    newimages_eye.append(data_preprocess(cv2.addWeighted(eye_x[i],1,validimages_eye[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_eye = np.asarray(newimages_eye)
  uniquelabels_eye.append(len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_eye), axis=1))))
print("Done")

Eyebrows Poison Image Test In Progress
Eye: 0.0%, Time: 2020-12-21 17:22:32.567732
Eye: 4.87%, Time: 2020-12-21 17:23:32.245963
Eye: 9.74%, Time: 2020-12-21 17:24:32.808744
Eye: 14.61%, Time: 2020-12-21 17:25:33.340797
Eye: 19.49%, Time: 2020-12-21 17:26:32.957390
Eye: 24.36%, Time: 2020-12-21 17:27:32.896653
Eye: 29.23%, Time: 2020-12-21 17:28:32.577192
Eye: 34.1%, Time: 2020-12-21 17:29:32.572897
Eye: 38.97%, Time: 2020-12-21 17:30:32.940062
Eye: 43.84%, Time: 2020-12-21 17:31:33.085212
Eye: 48.71%, Time: 2020-12-21 17:32:33.204530
Eye: 53.59%, Time: 2020-12-21 17:33:32.697732
Eye: 58.46%, Time: 2020-12-21 17:34:31.098423
Eye: 63.33%, Time: 2020-12-21 17:35:29.682495
Eye: 68.2%, Time: 2020-12-21 17:36:28.503668
Eye: 73.07%, Time: 2020-12-21 17:37:27.857875
Eye: 77.94%, Time: 2020-12-21 17:38:27.053759
Eye: 82.81%, Time: 2020-12-21 17:39:26.451703
Eye: 87.69%, Time: 2020-12-21 17:40:25.907754
Eye: 92.56%, Time: 2020-12-21 17:41:25.943204
Eye: 97.43%, Time: 2020-12-21 17:42:25.879976
D

In [ ]:
#lipstick poison image check
uniquelabels_lip = []
print("Lipstick Poison Image Test In Progress")
for i in range(len(lip_x)):
  if i%500 == 0:
    print("Lip: " + str(round((i/len(lip_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_lip = []
  validimages_lip = []
  for j in range(numoverlays):
    validimages_lip.append(valid_x[choice(sequence)])
    newimages_lip.append(data_preprocess(cv2.addWeighted(lip_x[i],1,validimages_lip[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_lip = np.asarray(newimages_lip)
  uniquelabels_lip.append(len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_lip), axis=1))))
print("Done")

Lipstick Poison Image Test In Progress
Lip: 0.0%, Time: 2020-12-21 17:42:57.430068
Lip: 4.87%, Time: 2020-12-21 17:43:57.765161
Lip: 9.74%, Time: 2020-12-21 17:44:57.361389
Lip: 14.61%, Time: 2020-12-21 17:45:56.278735
Lip: 19.49%, Time: 2020-12-21 17:46:55.465111
Lip: 24.36%, Time: 2020-12-21 17:47:54.650265
Lip: 29.23%, Time: 2020-12-21 17:48:53.632814
Lip: 34.1%, Time: 2020-12-21 17:49:52.705425
Lip: 38.97%, Time: 2020-12-21 17:50:51.728340
Lip: 43.84%, Time: 2020-12-21 17:51:50.824921
Lip: 48.71%, Time: 2020-12-21 17:52:49.871296
Lip: 53.59%, Time: 2020-12-21 17:53:48.945961
Lip: 58.46%, Time: 2020-12-21 17:54:47.831225
Lip: 63.33%, Time: 2020-12-21 17:55:46.928716
Lip: 68.2%, Time: 2020-12-21 17:56:46.304589
Lip: 73.07%, Time: 2020-12-21 17:57:45.877457
Lip: 77.94%, Time: 2020-12-21 17:58:45.264877
Lip: 82.81%, Time: 2020-12-21 17:59:44.731019
Lip: 87.69%, Time: 2020-12-21 18:00:44.633652
Lip: 92.56%, Time: 2020-12-21 18:01:44.126642
Lip: 97.43%, Time: 2020-12-21 18:02:43.320067
D

In [ ]:
#sunglasses poison image check
uniquelabels_sun = []
print("Sunglasses Poison Image Test In Progress")
for i in range(len(sun_x)):
  if i%500 == 0:
    print("Sun: " + str(round((i/len(sun_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_sun = []
  validimages_sun = []
  for j in range(numoverlays):
    validimages_sun.append(valid_x[choice(sequence)])
    newimages_sun.append(data_preprocess(cv2.addWeighted(sun_x[i],1,validimages_sun[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_sun = np.asarray(newimages_sun)
  uniquelabels_sun.append(len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_sun), axis=1))))
print("Done")

Sunglasses Poison Image Test In Progress
Sun: 0.0%, Time: 2020-12-21 18:03:14.689910
Sun: 4.87%, Time: 2020-12-21 18:04:13.653642
Sun: 9.74%, Time: 2020-12-21 18:05:12.579382
Sun: 14.61%, Time: 2020-12-21 18:06:11.659260
Sun: 19.49%, Time: 2020-12-21 18:07:10.839080
Sun: 24.36%, Time: 2020-12-21 18:08:09.864760
Sun: 29.23%, Time: 2020-12-21 18:09:08.876644
Sun: 34.1%, Time: 2020-12-21 18:10:08.708128
Sun: 38.97%, Time: 2020-12-21 18:11:08.317827
Sun: 43.84%, Time: 2020-12-21 18:12:07.784426
Sun: 48.71%, Time: 2020-12-21 18:13:07.278211
Sun: 53.59%, Time: 2020-12-21 18:14:06.318500
Sun: 58.46%, Time: 2020-12-21 18:15:05.666433
Sun: 63.33%, Time: 2020-12-21 18:16:05.270934
Sun: 68.2%, Time: 2020-12-21 18:17:04.409709
Sun: 73.07%, Time: 2020-12-21 18:18:04.198394
Sun: 77.94%, Time: 2020-12-21 18:19:03.313431
Sun: 82.81%, Time: 2020-12-21 18:20:02.766793
Sun: 87.69%, Time: 2020-12-21 18:21:02.178555
Sun: 92.56%, Time: 2020-12-21 18:22:01.586658
Sun: 97.43%, Time: 2020-12-21 18:23:01.386653

In [ ]:
#Two standard deviations accounts for most data
eyethr = np.mean(uniquelabels_eye) + 2*(np.std(uniquelabels_eye))
lipthr = np.mean(uniquelabels_lip) + 2*(np.std(uniquelabels_lip))
sunthr = np.mean(uniquelabels_sun) + 2*(np.std(uniquelabels_sun))

MTMTthr = round((eyethr + lipthr + sunthr)/3)

print("MTMT Threshold:", MTMTthr)

MTMT Threshold: 42


### Applying Threshold to Model

In [ ]:
#IMPORTANT: line below should be commented out if threshold calculation section is run
threshold = 21
anon1thr = 46
MTMTthr = 42

#### Sunglasses Model

In [ ]:
#clean test images
print("Test Image Check In Progress")
test_yhat = []
for i in range(len(test_x)):
  if i%500 == 0:
    print("Test: " + str(round((i/len(test_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_test = []
  validimages_test = []
  for j in range(numoverlays):
    validimages_test.append(valid_x[choice(sequence)])
    newimages_test.append(data_preprocess(cv2.addWeighted(test_x[i],1,validimages_test[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_test = np.asarray(newimages_test)
  uniquevals = len(np.unique(np.argmax(model_BadNetSTRIP.predict(newimagesnda_test), axis=1)))
  if uniquevals > threshold: #clean
    test_yhat.append(1)
  else:
    test_yhat.append(0)

print("")
truepos = round((sum(test_yhat)/len(test_yhat))*100, 2)
print("True Pos: " + str(truepos) + "%")

Test Image Check In Progress
Test: 0.0%, Time: 2020-12-17 16:54:59.922677
Test: 3.9%, Time: 2020-12-17 16:56:03.979307
Test: 7.79%, Time: 2020-12-17 16:57:07.826514
Test: 11.69%, Time: 2020-12-17 16:58:11.744190
Test: 15.59%, Time: 2020-12-17 16:59:15.296441
Test: 19.49%, Time: 2020-12-17 17:00:19.478080
Test: 23.38%, Time: 2020-12-17 17:01:23.291036
Test: 27.28%, Time: 2020-12-17 17:02:26.836840
Test: 31.18%, Time: 2020-12-17 17:03:31.002317
Test: 35.07%, Time: 2020-12-17 17:04:34.041122
Test: 38.97%, Time: 2020-12-17 17:05:38.589887
Test: 42.87%, Time: 2020-12-17 17:06:42.822844
Test: 46.77%, Time: 2020-12-17 17:07:47.070076
Test: 50.66%, Time: 2020-12-17 17:08:51.322822
Test: 54.56%, Time: 2020-12-17 17:09:55.258280
Test: 58.46%, Time: 2020-12-17 17:10:59.713888
Test: 62.35%, Time: 2020-12-17 17:12:03.934405
Test: 66.25%, Time: 2020-12-17 17:13:07.992974
Test: 70.15%, Time: 2020-12-17 17:14:11.157693
Test: 74.05%, Time: 2020-12-17 17:15:15.027998
Test: 77.94%, Time: 2020-12-17 17:16

In [ ]:
#poisoned images
print("Poisoned Image Check In Progress")
poison_yhat = []
for i in range(len(poison_x)):
  if i%500 == 0:
    print("Poison: " + str(round((i/len(poison_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_poison = []
  validimages_poison = []
  for j in range(numoverlays):
    validimages_poison.append(valid_x[choice(sequence)])
    newimages_poison.append(data_preprocess(cv2.addWeighted(poison_x[i],1,validimages_poison[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_poison = np.asarray(newimages_poison)
  uniquevals = len(np.unique(np.argmax(model_BadNetSTRIP.predict(newimagesnda_poison), axis=1)))
  if uniquevals > threshold: #clean
    poison_yhat.append(0) #swapped 0 and 1 for poisoned images
  else:
    poison_yhat.append(1)

print("")
trueneg = round((sum(poison_yhat)/len(poison_yhat))*100, 2)
print("True Neg: " + str(trueneg) + "%")

Poisoned Image Check In Progress
Poison: 0.0%, Time: 2020-12-17 17:54:02.986163
Poison: 3.9%, Time: 2020-12-17 17:55:06.473590
Poison: 7.79%, Time: 2020-12-17 17:56:10.044548
Poison: 11.69%, Time: 2020-12-17 17:57:13.909583
Poison: 15.59%, Time: 2020-12-17 17:58:17.617518
Poison: 19.49%, Time: 2020-12-17 17:59:21.142095
Poison: 23.38%, Time: 2020-12-17 18:00:25.416868
Poison: 27.28%, Time: 2020-12-17 18:01:29.442170
Poison: 31.18%, Time: 2020-12-17 18:02:33.419858
Poison: 35.07%, Time: 2020-12-17 18:03:37.556310
Poison: 38.97%, Time: 2020-12-17 18:04:40.655970
Poison: 42.87%, Time: 2020-12-17 18:05:44.736556
Poison: 46.77%, Time: 2020-12-17 18:06:48.443800
Poison: 50.66%, Time: 2020-12-17 18:07:51.115151
Poison: 54.56%, Time: 2020-12-17 18:08:54.212408
Poison: 58.46%, Time: 2020-12-17 18:09:57.489914
Poison: 62.35%, Time: 2020-12-17 18:11:00.467976
Poison: 66.25%, Time: 2020-12-17 18:12:04.676836
Poison: 70.15%, Time: 2020-12-17 18:13:08.517286
Poison: 74.05%, Time: 2020-12-17 18:14:11

#### Anonymous 1 Model

In [ ]:
# anonymous 1 poisoned images
print("Anonymous 1 Poisoned Image Check In Progress")
anon1_yhat = []
for i in range(len(anon1_x)):
  if i%500 == 0:
    print("Anon 1: " + str(round((i/len(anon1_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_anon1 = []
  validimages_anon1 = []
  for j in range(numoverlays):
    validimages_anon1.append(valid_x[choice(sequence)])
    newimages_anon1.append(data_preprocess(cv2.addWeighted(anon1_x[i],1,validimages_anon1[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_anon1 = np.asarray(newimages_anon1)
  uniquevals = len(np.unique(np.argmax(model_BadNetAnon1STRIP.predict(newimagesnda_anon1), axis=1)))
  if uniquevals > anon1thr: #clean
    anon1_yhat.append(0) #swapped 0 and 1 for poisoned images
  else:
    anon1_yhat.append(1)

print("")
trueneg = round((sum(anon1_yhat)/len(anon1_yhat))*100, 2)
print("True Neg: " + str(trueneg) + "%")

Anonymous 1 Poisoned Image Check In Progress
Anon 1: 0.0%, Time: 2020-12-21 17:01:30.753254
Anon 1: 4.87%, Time: 2020-12-21 17:02:31.000606
Anon 1: 9.74%, Time: 2020-12-21 17:03:30.750553
Anon 1: 14.61%, Time: 2020-12-21 17:04:31.098376
Anon 1: 19.49%, Time: 2020-12-21 17:05:32.242396
Anon 1: 24.36%, Time: 2020-12-21 17:06:32.563126
Anon 1: 29.23%, Time: 2020-12-21 17:07:32.870157
Anon 1: 34.1%, Time: 2020-12-21 17:08:32.788262
Anon 1: 38.97%, Time: 2020-12-21 17:09:32.868242
Anon 1: 43.84%, Time: 2020-12-21 17:10:33.266583
Anon 1: 48.71%, Time: 2020-12-21 17:11:33.484605
Anon 1: 53.59%, Time: 2020-12-21 17:12:34.152862
Anon 1: 58.46%, Time: 2020-12-21 17:13:34.532085
Anon 1: 63.33%, Time: 2020-12-21 17:14:34.636637
Anon 1: 68.2%, Time: 2020-12-21 17:15:35.049385
Anon 1: 73.07%, Time: 2020-12-21 17:16:35.659834
Anon 1: 77.94%, Time: 2020-12-21 17:17:35.910545
Anon 1: 82.81%, Time: 2020-12-21 17:18:36.420228
Anon 1: 87.69%, Time: 2020-12-21 17:19:36.395192
Anon 1: 92.56%, Time: 2020-12-

#### MTMT Model

In [ ]:
# eyebrow poisoned images
print("Eyebrow Poisoned Image Check In Progress")
eye_yhat = []
for i in range(len(eye_x)):
  if i%500 == 0:
    print("Eye: " + str(round((i/len(eye_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_eye = []
  validimages_eye = []
  for j in range(numoverlays):
    validimages_eye.append(valid_x[choice(sequence)])
    newimages_eye.append(data_preprocess(cv2.addWeighted(eye_x[i],1,validimages_eye[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_eye = np.asarray(newimages_eye)
  uniquevals = len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_eye), axis=1)))
  if uniquevals > MTMTthr: #clean
    eye_yhat.append(0) #swapped 0 and 1 for poisoned images
  else:
    eye_yhat.append(1)

print("")
trueneg = round((sum(eye_yhat)/len(eye_yhat))*100, 2)
print("True Neg: " + str(trueneg) + "%")

Eyebrow Poisoned Image Check In Progress
Eye: 0.0%, Time: 2020-12-21 18:25:19.751697
Eye: 4.87%, Time: 2020-12-21 18:26:19.278498
Eye: 9.74%, Time: 2020-12-21 18:27:18.975858
Eye: 14.61%, Time: 2020-12-21 18:28:18.834584
Eye: 19.49%, Time: 2020-12-21 18:29:18.320816
Eye: 24.36%, Time: 2020-12-21 18:30:17.561637
Eye: 29.23%, Time: 2020-12-21 18:31:18.743654
Eye: 34.1%, Time: 2020-12-21 18:32:18.699774
Eye: 38.97%, Time: 2020-12-21 18:33:18.496293
Eye: 43.84%, Time: 2020-12-21 18:34:18.232884
Eye: 48.71%, Time: 2020-12-21 18:35:18.255758
Eye: 53.59%, Time: 2020-12-21 18:36:17.591274
Eye: 58.46%, Time: 2020-12-21 18:37:17.404054
Eye: 63.33%, Time: 2020-12-21 18:38:17.384530
Eye: 68.2%, Time: 2020-12-21 18:39:17.254424
Eye: 73.07%, Time: 2020-12-21 18:40:16.452122
Eye: 77.94%, Time: 2020-12-21 18:41:15.909125
Eye: 82.81%, Time: 2020-12-21 18:42:15.093682
Eye: 87.69%, Time: 2020-12-21 18:43:14.635784
Eye: 92.56%, Time: 2020-12-21 18:44:14.362629
Eye: 97.43%, Time: 2020-12-21 18:45:13.107136

In [ ]:
# lipstick poisoned images
print("Lipstick Poisoned Image Check In Progress")
lip_yhat = []
for i in range(len(lip_x)):
  if i%500 == 0:
    print("Lip: " + str(round((i/len(lip_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_lip = []
  validimages_lip = []
  for j in range(numoverlays):
    validimages_lip.append(valid_x[choice(sequence)])
    newimages_lip.append(data_preprocess(cv2.addWeighted(lip_x[i],1,validimages_lip[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_lip = np.asarray(newimages_lip)
  uniquevals = len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_lip), axis=1)))
  if uniquevals > MTMTthr: #clean
    lip_yhat.append(0) #swapped 0 and 1 for poisoned images
  else:
    lip_yhat.append(1)

print("")
trueneg = round((sum(lip_yhat)/len(lip_yhat))*100, 2)
print("True Neg: " + str(trueneg) + "%")

Lipstick Poisoned Image Check In Progress
Lip: 0.0%, Time: 2020-12-21 19:16:57.748395
Lip: 4.87%, Time: 2020-12-21 19:17:56.825906
Lip: 9.74%, Time: 2020-12-21 19:18:55.874253
Lip: 14.61%, Time: 2020-12-21 19:19:54.926428
Lip: 19.49%, Time: 2020-12-21 19:20:53.869762
Lip: 24.36%, Time: 2020-12-21 19:21:52.881175
Lip: 29.23%, Time: 2020-12-21 19:22:52.332120
Lip: 34.1%, Time: 2020-12-21 19:23:51.432658
Lip: 38.97%, Time: 2020-12-21 19:24:50.843458
Lip: 43.84%, Time: 2020-12-21 19:25:50.441751
Lip: 48.71%, Time: 2020-12-21 19:26:49.770533
Lip: 53.59%, Time: 2020-12-21 19:27:49.137655
Lip: 58.46%, Time: 2020-12-21 19:28:49.004883
Lip: 63.33%, Time: 2020-12-21 19:29:48.316476
Lip: 68.2%, Time: 2020-12-21 19:30:47.410982
Lip: 73.07%, Time: 2020-12-21 19:31:46.170812
Lip: 77.94%, Time: 2020-12-21 19:32:45.716037
Lip: 82.81%, Time: 2020-12-21 19:33:44.858216
Lip: 87.69%, Time: 2020-12-21 19:34:43.981483
Lip: 92.56%, Time: 2020-12-21 19:35:42.615117
Lip: 97.43%, Time: 2020-12-21 19:36:41.01948

In [ ]:
# sunglasses poisoned images
print("Sunglasses Poisoned Image Check In Progress")
sun_yhat = []
for i in range(len(sun_x)):
  if i%500 == 0:
    print("Sun: " + str(round((i/len(sun_x))*100, 2)) + "%, Time: " + str(datetime.datetime.now()))
  newimages_sun = []
  validimages_sun = []
  for j in range(numoverlays):
    validimages_sun.append(valid_x[choice(sequence)])
    newimages_sun.append(data_preprocess(cv2.addWeighted(sun_x[i],1,validimages_sun[j],1,0,dtype=cv2.CV_64FC3)))

  newimagesnda_sun = np.asarray(newimages_sun)
  uniquevals = len(np.unique(np.argmax(model_BadNetMTMTSTRIP.predict(newimagesnda_sun), axis=1)))
  if uniquevals > MTMTthr: #clean
    sun_yhat.append(0) #swapped 0 and 1 for poisoned images
  else:
    sun_yhat.append(1)

print("")
trueneg = round((sum(sun_yhat)/len(sun_yhat))*100, 2)
print("True Neg: " + str(trueneg) + "%")

Sunglasses Poisoned Image Check In Progress
Sun: 0.0%, Time: 2020-12-21 18:45:47.281259
Sun: 4.87%, Time: 2020-12-21 18:46:47.019705
Sun: 9.74%, Time: 2020-12-21 18:47:46.065082
Sun: 14.61%, Time: 2020-12-21 18:48:45.319170
Sun: 19.49%, Time: 2020-12-21 18:49:44.549264
Sun: 24.36%, Time: 2020-12-21 18:50:43.438351
Sun: 29.23%, Time: 2020-12-21 18:51:42.709419
Sun: 34.1%, Time: 2020-12-21 18:52:42.437367
Sun: 38.97%, Time: 2020-12-21 18:53:41.905485
Sun: 43.84%, Time: 2020-12-21 18:54:41.322596
Sun: 48.71%, Time: 2020-12-21 18:55:40.818949
Sun: 53.59%, Time: 2020-12-21 18:56:40.631029
Sun: 58.46%, Time: 2020-12-21 18:57:39.953483
Sun: 63.33%, Time: 2020-12-21 18:58:38.912542
Sun: 68.2%, Time: 2020-12-21 18:59:38.250303
Sun: 73.07%, Time: 2020-12-21 19:00:37.598696
Sun: 77.94%, Time: 2020-12-21 19:01:36.334049
Sun: 82.81%, Time: 2020-12-21 19:02:35.240465
Sun: 87.69%, Time: 2020-12-21 19:03:34.375124
Sun: 92.56%, Time: 2020-12-21 19:04:33.247403
Sun: 97.43%, Time: 2020-12-21 19:05:32.170